In [1]:
import pandas as pd
import numpy as np

url = '.\대출금리\중소기업 대출\기업대출금리(마통-중소기업).csv'
class read():
    credit_distribution = pd.read_csv('.\기업신용등급분포.csv', encoding='cp949')
    cd = np.array(credit_distribution['구성 비율']).astype('float')
    

    def __init__(self,url):
        self.loan_dev = url

    def tidy(self):
        self.loan_dev = pd.read_csv(self.loan_dev,encoding='cp949')
        self.loan_dev['은행'] = self.loan_dev['은행'].str.strip()
        self.loan_dev.columns = ['은행','구분','1~3등급','4등급','5등급','6등급','7~10등급','평균금리','참고사항']
        self.loan_dev.drop([0],axis=0,inplace=True)
        self.loan_dev.drop(['참고사항','구분'],axis=1,inplace=True)
        self.loan_dev['평균금리']=self.loan_dev['평균금리'].replace('-',np.NaN)
        self.loan_dev.dropna(axis=0,inplace=True)
        self.loan_dev.reset_index(drop=True,inplace=True)
        self.loan_dev.replace('-',0,inplace=True)
        self.interest_grade = self.loan_dev.loc[:,'1~3등급':'7~10등급']
        return self.loan_dev

    def multi(self):
        self.a = []
        for i in range(len(self.interest_grade)):
            self.rate_grade = np.array(self.interest_grade.loc[i:i,'1~3등급':'7~10등급'], dtype='float')[0]
            #은행 상품 단위 신용 등급 별 이자율
            self.boolean = self.rate_grade.astype('bool')
            #상품의 대상 고객 
            self.ac_cd = sum(self.boolean * self.cd)
            # 전체 신용 등급 대비 타겟 고객 비율
            self.customer_ration = self.cd / self.ac_cd
            # 타겟 고객 대비 특정 신용 등급 고객 비율
            self.real_income = self.rate_grade @ self.customer_ration
            # 대출 금리에 대한 실제 고객의 가중 평균으로 구한 상품 수익률
            self.a.append(self.real_income)
        
        self.loan_dev['가중평균수익'] = self.a
        return self.loan_dev


a = read(url)
a.tidy()
a.multi()

,은행,1~3등급,4등급,5등급,6등급,7~10등급,평균금리,가중평균수익
0,BNK경남은행,4.47,4.62,5.38,5.89,6.30,4.67,5.630059
1,BNK부산은행,4.03,4.46,4.96,6.16,7.72,4.39,5.647787
2,DGB대구은행,4.52,4.94,5.67,7.28,10.91,5.75,6.604920
3,IBK기업은행,4.55,5.27,6.91,9.34,8.56,5.89,8.227765
4,KB국민은행,3.69,4.67,6.16,9.02,9.66,5.30,7.753098
5,KDB산업은행,2.93,3.73,4.68,6.14,10.01,3.73,5.499875
6,NH농협은행,3.62,4.00,4.52,7.29,7.23,4.20,6.141661
7,SH수협은행,4.46,5.07,4.02,9.72,9.50,4.59,7.519735
8,광주은행,4.77,5.46,5.39,6.50,9.15,5.51,6.068231
9,스탠다드차타드은행,0,0,0,0,3.34,3.34,3.340000
